# "Linked Art API: 超文本应用语言链接"

> **上级页面**: ["Linked Art API 1.0"]("/api/1.0/")

---



In [ ]:
# 环境设置
from cromulent import model, vocab

# 设置 base_url 以获得更清晰的输出
model.factory.base_url = 'http://test.com/museum/'





## 简介

Linked Art 使用名为 [JSON 超文本应用语言](https://datatracker.ietf.org/doc/html/draft-kelly-json-hal-11)（HAL）的标准在关联数据响应中包含非语义数据。这使得记录能够自文档化，说明所用模型、API 和任何扩展的版本，并引用相同信息或相关 API 的其他格式。


## 超文本应用语言

HAL 定义了一个名为 `_links` 的属性，它是一个包含命名链接的 JSON 对象。链接表示为 JSON 对象，其中包含包含 URI 的 `href` 属性，以及一些额外的可选字段。这个结构嵌入到 JSON-LD 记录中，但不是语义的。因此，如果记录被转换为 RDF 图然后重新序列化，这个 API 级别的数据将**不会**出现在图或新的序列化中。

最简单的 HAL 块只是对记录本身的引用，使用 `self` 键：



In [ ]:
{
  "_links": {
    "self": "http://example.org/linked_art/objects/1234"
  }
}




可以通过添加命名空间属性来扩展链接类型，允许 Linked Art API 和本地扩展定义客户端理解和遵循的关系。命名空间在一个名为 `curies` 的块中给出，这是一个链接数组，带有额外的 `name` 属性给出前缀，以及值为 `true` 的 `templated` 属性，让客户端知道它是一个 URL 模板而不是完整的 URL。如果使用 `curies` 块中给出的模板扩展，链接将解析为其语义和用途的描述。`_links` 块可能像这样开始，可能在 `curies` 中有额外的条目用于本地命名和定义的扩展：



In [ ]:
{
  "_links": {
    "self": "http://example.org/linked_art/objects/1234"
    "curies": [
      {"name":"la","href":"https://linked.art/api/rels/1/{rel}","templated":true}
    ]
  }
}




## 版本控制

Linked Art API 定义了三个核心属性来处理 API 版本控制。

版本控制属性包括：

* `la:modelVersion` - 记录使用的 linked art 模型版本，可能与 API 版本不同（必需）
* `la:apiVersion` - 记录使用的 linked art API 版本（必需）
* `la:localVersion` - 记录使用的任何扩展或扩展集的版本（可选）

`href` 字段中给出的 URI 必须引用被引用版本的易读文档。`name` 字段必须存在且必须具有以下结构：

`v{major-version-integer}.{minor-version-integer}.{patch-version-string}`

因此，客户端可以获取 name 字段，删除前导的"v"字符，然后在'.'字符上分割结果以获取主版本、次版本和补丁版本。主版本和次版本必须是整数，以便客户端可以将它们从字符串转换为数字并比较 10 大于 2（例如）。补丁组件是一个字符串，不得包含'.'字符，并且必须正确地作为字符串从最低到最高排序，使得"patch-10"出现在"patch-02"之后。

这将允许客户端确定它们是否能够处理记录。如果主版本高于客户端支持的主版本号，那么不太可能。如果主版本号等于或低于客户端支持的版本，那么客户端应该能够处理信息。如果相对于主版本的次版本号高于支持的版本，那么信息可能会丢失，但次版本在相应主版本内的所有其他次版本向后兼容。补丁不引入新的或修改的功能，因此它们主要用于提高意识。

`_links` 部分现在可能看起来像以下内容：



In [ ]:
{
  "_links": {
    "self": "http://example.org/linked_art/objects/1234"
    "curies": [
      {"name":"la","href":"https://linked.art/api/rels/1/{rel}","templated":true}
    ],
    "la:modelVersion": {"href":"https://linked.art/model/1.0/", "name": "v1.0"},
    "la:apiVersion": {"href":"https://linked.art/api/1.0/", "name": "v1.0"},
    "la:localVersion": {"href":"https://example.org/extension/maps/geo-210", "name": "v2.1.0-beta-01"},
  }
}




请注意，作为命名空间，Linked Art 关系的 curie 模板不包括次版本号，因为次版本必须与主版本向后兼容。未来的主版本将引入一组新的链接（尽管可能与之前的版本相同），并且可能还会改变遵循这些链接的响应格式。


## 链接到其他表示

有两个 HAL 链接可用于引用 API 生态系统的其他部分。特别是，从 JSON-LD 数据链接到为人类呈现数据的网页、到不同关联数据格式中的其他表示、以及到 IIIF 变更发现 API 端点（这将允许远程系统收集并保持与整体数据集同步）是非常有价值的。

使用 `alternate` 属性可以实现从 JSON-LD 记录链接到数据的呈现，而不是关于对象的外部网页集合。为了传达呈现的媒体类型是 HTML，链接对象必须具有值为 `text/html` 的 `type` 属性。`alternate` 属性也可用于引用数据的其他表示，例如 Turtle 中的语义表示，或 CSV 或 XML 等非语义表示。`type` 属性必须存在，并具有该格式的正确媒体类型。

同样可以使用 `collection` 属性引用 [IIIF 变更发现端点](../discovery/)。

因此，扩展的 `_links` 部分可能看起来像：



In [ ]:
{
  "_links": {
    "self": "http://example.org/linked_art/objects/1234"
    "curies": [
      {"name":"la","href":"https://linked.art/api/rels/1/{rel}","templated":true}
    ],

    "alternate": [
      {"href": "https://example.org/view/1234.html", "type": "text/html"},
      {"href": "https://example.org/linked_art/objects/1234.ttl", "type": "text/turtle"}
    ],
    "collection": {"href": "https://example.org/discovery/collection.json"},

    "la:modelVersion": {"href":"https://linked.art/model/1.0/", "name": "v1.0"},
    "la:apiVersion": {"href":"https://linked.art/api/1.0/", "name": "v1.0"},
    "la:localVersion": {"href":"https://example.org/extension/maps/geo-210", "name": "v2.1.0-beta-01"},

  }
}




## 链接到搜索

在 Linked Art 中，HAL 链接的主要用途是将消费应用程序定向到与当前记录相关的搜索结果。这在 [搜索](../search/) API 中有记录。